<a href="https://qworld.net" target="_blank" align="left"><img src="../qworld/images/header.jpg"  align="left"></a>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $
$ \newcommand{\greenbit}[1] {\mathbf{{\color{green}#1}}} $
$ \newcommand{\bluebit}[1] {\mathbf{{\color{blue}#1}}} $
$ \newcommand{\redbit}[1] {\mathbf{{\color{red}#1}}} $
$ \newcommand{\brownbit}[1] {\mathbf{{\color{brown}#1}}} $
$ \newcommand{\blackbit}[1] {\mathbf{{\color{black}#1}}} $

<font style="font-size:28px;" align="left"><b>Quantum Tomography  </b></font>
<br>
_prepared by Abuzer Yakaryilmaz_

_ProjectQ adaptation by Vishal Sharathchandra Bajpe and Marija Šćekić_
<br><br>


We study a simplified version of quantum tomography here. 

It is similar to learn the bias of a coin by collecting statistics from tossing this coin many times. 

However, only making measurement may not be enough to make a correct estimation in quantum case.

Suppose that you are given 1000 copies of a qubit and your task is to learn the state of this qubit. We use a python class called "unknown_qubit" for doing our quantum experiments. 

Please run the following cell before continuing.

In [ ]:
from random import randrange
from math import pi
from projectq import MainEngine
from projectq.ops import H, Ry, Measure, All
from projectq.backends import Simulator, CircuitDrawerMatplotlib
from projectq.setups.default import get_engine_list

# class unknown_qubit
#   available_qubits = 1000 -> you can get at most 1000 qubit copies
#   get_qubits(number_of_qubits) -> you get the specified number of active qubits for your experiments
#   rotate(angle) -> each active qubit is rotated by Ry(2*angle)
#   applyH() -> Hadamard operator is applied to each active qubit
#   measure() -> each active qubit is measured and the outcomes are printed and as well as returned as a dictionary
#             -> after the measurements, these qubits are destroyed   
#   compare_my_guess(my_angle) -> your guess in radian is compared with the actual angle in radian

class unknown_qubit:
    def __init__(self):
        self.__theta = randrange(18000)/18000*pi                  
        self.__available_qubits = 1000
        self.__active_qubits = 0
        print(self.__available_qubits,"qubits are created")
        self.__operations = []
    
    def get_qubits(self, number_of_qubits=None):
        if number_of_qubits is None or isinstance(number_of_qubits,int) is False or number_of_qubits < 1:
            print()
            print("ERROR: the method 'get_qubits' takes the number of qubit(s) as a positive integer, i.e., get_qubits(100)")
        elif number_of_qubits <= self.__available_qubits:            
            self.__active_qubits = number_of_qubits
            self.__available_qubits = self.__available_qubits - self.__active_qubits
            print()
            print("You have",number_of_qubits,"active qubits that are set to (cos(theta),sin(theta))")
            self.__operations = [['Ry',self.__theta]]
            self.available_qubits()
        else:
            print()
            print("WARNING: you requested",number_of_qubits,"qubits, but there is not enough available qubits!")
            self.available_qubits()
            
    def rotate(self,angle=None):
        if angle is None or (isinstance(angle,float) is False and isinstance(angle,int) is False):
            print()
            print("ERROR: the method 'rotate_qubits' takes a real-valued angle in radian as its parameter, i.e., rotate_qubits(1.2121)")
        elif self.__active_qubits > 0:
            self.__operations.append(["Ry",angle])
            print()
            print("your active qubits are rotated by angle",angle,"in radian")
        else:
            print()
            print("WARNING: there is no active qubits -- you might first execute 'get_qubits()' method")
            self.available_qubits()
            
    def applyH(self):
        if self.__active_qubits > 0:
            self.__operations.append(["H",0])
            print()
            print("Hadamard operator is applied to each of the active qubit")
        else:
            print()
            print("WARNING: there is no active qubits -- you may first execute 'get_qubits()' method")
            self.available_qubits()
    def measure(self):
        if self.__active_qubits > 0:
            iteration = self.__active_qubits # each active qubit will be executed
            # create a dictionary to store measurement outcomes
            outcomes = {}
            for _ in range(iteration): # the same circuit will be executed in each iteration
                qdrawer = CircuitDrawerMatplotlib()
                qengine = MainEngine(backend = Simulator(), engine_list = [qdrawer]+get_engine_list())
                qubit = qengine.allocate_qureg(1) # create the qubit
                for op in self.__operations: # apply every defined operator one by one
                    if op[0]=="H": H | qubit[0] # Hadamard operator
                    elif op[0]=="Ry": Ry(2*op[1]) | qubit [0] # Ry rotation
                qengine.flush() # flush the circuit to the backend
                Measure | qubit
                key = str(int(qubit[0])) # read the measurement outcome as a string
                if key in outcomes: outcomes[key]+=1 # increase the number of key by 1
                else: outcomes[key] = 1 # create the key with value 1  
            #qdrawer.draw() # for debugging
            print()
            print("your active",self.__active_qubits,"qubits are measured")
            print("outcomes =",outcomes)
            self.__active_qubits = 0
            return outcomes
        else:
            print()
            print("WARNING: there is no active qubits -- you may first execute 'get_qubits()' method")
            self.available_qubits()         
            
    def compare_my_guess(self,my_angle):
        if my_angle is None or (isinstance(my_angle,float) is False and isinstance(my_angle,int) is False):
            print("ERROR: the method 'compare_my_guess' takes a real-valued angle in radian as your guessed angle, i.e., compare_my_guess(1.2121)")
        else:
            self.__available_qubits = 0
            diff = abs(my_angle-self.__theta)
            print()
            print(self.__theta,"is the original",)
            print(my_angle,"is your guess")
            print("the angle difference between the original theta and your guess is",diff/pi*180,"degree")
            print("--> the number of available qubits is (set to) zero, and so you cannot make any further experiment")

    def available_qubits(self):
        print("--> the number of available unused qubit(s) is",self.__available_qubits)   

class unknown_qubit:
    
    available_qubits = 1000 -> you can get at most 1000 qubit copies
    get_qubits(number_of_qubits) -> you get the specified number of active qubits for your experiments
    rotate(angle) -> each active qubit is rotated by Ry(2*angle)
    applyH() -> Hadamard operator is applied to each active qubit
    measure() -> each active qubit is measured and the outcomes are printed and as well as returned as a dictionary
              -> after the measurements, these qubits are destroyed   
    compare_my_guess(my_angle) -> your guess in radian is compared with the actual angle in radian

<h3> Task 1 </h3>

You are given 1000 copies of the identical qubits which are in the same quantum state lying in the first or second quadrant of the unit circle. 

This quantum state is represented by an angle $ \theta \in [0,\pi) $, and your task is to estimate this angle.

You use the class __unknown_qubit__ and its methods for your experiments. 

_Remark that the measurement outcomes of the quantum states with angles $ \pi \over 3 $ and $ 2 \pi \over 3 $ are identical even though they are different quantum states. Therefore, getting 1000 qubits and then measuring them does not guarantee a correct estimation._

Test your solution at least ten times.

In [ ]:
from random import randrange
from math import pi
from projectq import MainEngine
from projectq.ops import Ry, Measure, All
from projectq.backends import Simulator, CircuitDrawerMatplotlib
from projectq.setups.default import get_engine_list

# an angle theta is randomly picked and it is fixed througout the experiment
exp = unknown_qubit() # variable for your experiments
#
# You may use one or more of the following methods in approriate order
#
# exp.get_qubits(number_of_qubits)
# exp.rotate(angle)
# exp.applyH()
# exp.measure()
# exp.compare_my_guess(my_angle)
#

In [ ]:
# Test your solution several times
for i in range(10):
    exp = unknown_qubit()
    #
    #
    #

[click for our solution](Q52_Quantum_Tomography_Solution.ipynb#task1)

<h3> Task 2 (extra) </h3>

You are given 1000 identical quantum systems with two qubits that are in states $ \myvector{\cos \theta_1 \\ \sin \theta_1} $ and $ \myvector{\cos \theta_2 \\ \sin \theta_2} $, where $ \theta_1,\theta_2 \in [0,\pi) $.

Your task is to guess the values of $ \theta_1 $ and $ \theta_2 $. 

Create a quantum circuit with two qubits. 

Randomly pick $\theta_1$ and $ \theta_2 $ and set the states of qubits respectively. (Do not use $ \theta_1 $ and $ \theta_2 $ except initializing the qubits.)

Do experiments (making measurements and/or applying basic quantum operators) with your circuit(s). You may create more than one circuit.

Assume that the total number of shots does not exceed 1000 throughout the whole experiment.

_Since you have two qubits, your measurement outcomes will be '00', '01', '10', and '11'._

In [ ]:
######################
# Enter your code here
######################

<h3> Task 3 (Discussion) </h3>

If the angle in Task 1 is picked in range $ [0,2\pi) $, then can we determine its quadrant correctly?

<h3> Global phase </h3>

Suppose that we have a qubit and its state is either $ \ket{0} $ or $ -\ket{0} $.

Is there any sequence of one-qubit gates such that we can measure different results after applying them?

All one-qubit gates are $ 2 \times 2 $ matrices, and their application is represented by a single matrix: $ A_n \cdot \cdots \cdot A_2 \cdot A_1 = A $.

By linearity, if $ A \ket{0} = \ket{u} $, then $ A (- \ket{0}) = -\ket{u} $. Thus, after measurement, the probabilities of observing state $ \ket{0} $ and state $ \ket{1} $ are the same for $ \ket{u} $ and $ -\ket{u} $. Therefore, we cannot distinguish them.

Even though the states $ \ket{0} $ and $ -\ket{0} $ are different mathematically, they are assumed as identical from the physical point of view. 

The minus sign in front of $ -\ket{0} $ is called as a global phase.

In general, a global phase can be a complex number with magnitude 1.